# Train YOLO Pose Model (Bee Dataset)

This notebook trains a YOLO pose estimation model using data in `./labeled-data` (YOLO v1.0 pose labels).

## 1) Install/Import Dependencies

In [1]:
# If needed, uncomment to install Ultralytics
# %pip install -q ultralytics pyyaml

import os
from pathlib import Path

from ultralytics import YOLO

## 2) Configure Paths

In [2]:
ROOT = Path.cwd()
DATA_DIR = ROOT / "labeled-data"
IMAGES_DIR = DATA_DIR / "images"
LABELS_DIR = DATA_DIR / "labels"
DATA_YAML = DATA_DIR / "data.yaml"

assert DATA_DIR.exists(), f"Missing dataset folder: {DATA_DIR}"
assert IMAGES_DIR.exists(), f"Missing images folder: {IMAGES_DIR}"
assert LABELS_DIR.exists(), f"Missing labels folder: {LABELS_DIR}"
assert DATA_YAML.exists(), f"Missing YAML file: {DATA_YAML}"

print(f"Workspace root: {ROOT}")
print(f"Dataset folder: {DATA_DIR}")

Workspace root: c:\Users\bee-ops\code\Choice-assay
Dataset folder: c:\Users\bee-ops\code\Choice-assay\labeled-data


## 3a) Remove images without labels

In [3]:
# Remove any images that don't have corresponding labels
for img_file in IMAGES_DIR.glob("*.png"):
    label_file = LABELS_DIR / img_file.with_suffix(".txt").name
    if not label_file.exists():
        print(f"Removing image without label: {img_file}")
        img_file.unlink()

## 3b) Redo autosplit if required

In [4]:
# Autosplit the labeled-data into train, test, and validation sets
from ultralytics.data.split import autosplit


def redo_autosplit():
    # Ensure that the autosplit is run in the labeled-data directory, but return to the base directory
    # afterwards so that YOLO runs properly
    path = Path.cwd()
    if path.parts[-1] != "labeled-data":
        labeled_data_path = path / "labeled-data"
        os.chdir(labeled_data_path)
    else:
        path = path.parent
    autosplit(path=".", weights=(0.8, 0.12, 0.08))
    os.chdir(path)


#redo_autosplit()

## 4) Train YOLO Pose

In [5]:
OUTPUT = ROOT/"ml_runs"/"pose"
PRETRAINED_WEIGHTS = OUTPUT / "bee_pose" / "weights" / "best.pt"
IMGSZ = 448

print(f"Using device training YOLO pose model: {PRETRAINED_WEIGHTS}")

model = YOLO(PRETRAINED_WEIGHTS)
results = model.train(data=str(DATA_YAML), epochs=300, imgsz=IMGSZ, project=OUTPUT)
results

Using device training YOLO pose model: c:\Users\bee-ops\code\Choice-assay\ml_runs\pose\bee_pose\weights\best.pt
New https://pypi.org/project/ultralytics/8.4.16 available  Update with 'pip install -U ultralytics'
Ultralytics 8.4.6  Python-3.12.10 torch-2.10.0+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
engine\trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\bee-ops\code\Choice-assay\labeled-data\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=448, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, l

ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000022226004770>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.0340

## 5) Validate and Export

In [6]:
metrics = model.val(data=str(DATA_YAML))
metrics

Ultralytics 8.4.6  Python-3.12.10 torch-2.10.0+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
YOLOv8n-pose summary (fused): 82 layers, 3,103,934 parameters, 0 gradients, 8.4 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 480.564.7 MB/s, size: 62.7 KB)
val: Scanning C:\Users\bee-ops\code\Choice-assay\labeled-data\labels.cache... 49 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 49/49  0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 1.3it/s 3.1s1.4s
                   all         49         49      0.688       0.49      0.524      0.326      0.775      0.562      0.615      0.385
Speed: 0.9ms preprocess, 48.1ms inference, 0.0ms loss, 0.6ms postprocess per image
Results saved to C:\Users\bee-ops\code\expidite\runs\pose\val


ultralytics.utils.metrics.PoseMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002220181C770>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(P)', 'F1-Confidence(P)', 'Precision-Confidence(P)', 'Recall-Confidence(P)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.0340

In [8]:
best_model = Path("./ml_runs/pose/bee_pose/weights/best.pt")
print("Best model path:", best_model)

# Optional exports:
model.export(format='ncnn')
# model.export(format='torchscript')

Best model path: ml_runs\pose\bee_pose\weights\best.pt
Ultralytics 8.4.6  Python-3.12.10 torch-2.10.0+cpu CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)

PyTorch: starting from 'C:\Users\bee-ops\code\Choice-assay\ml_runs\pose\bee_pose\weights\best.pt' with input shape (1, 3, 448, 448) BCHW and output shape(s) (1, 26, 4116) (6.2 MB)
requirements: Ultralytics requirement ['ncnn'] not found, attempting AutoUpdate...
   ---------------------------------------- 0.0/3.8 MB ? eta -:--:--
   ------------- -------------------------- 1.3/3.8 MB 7.4 MB/s eta 0:00:01
   ------------------------------ --------- 2.9/3.8 MB 7.6 MB/s eta 0:00:01
   ------------------------------------ --- 3.4/3.8 MB 5.8 MB/s eta 0:00:01
   ---------------------------------------- 3.8/3.8 MB 6.1 MB/s  0:00:00

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip

requirements: AutoUpdate success  3.2s
WARNING requirements: Restart runtime or rerun 

'C:\\Users\\bee-ops\\code\\Choice-assay\\ml_runs\\pose\\bee_pose\\weights\\best_ncnn_model'